In [8]:
#Changes: 
    #- The script automatically generates a folder for harvest jsons. 
    #- The script generates a json called missing where all the skipped IDs are entered no matter where in the script things went wrong.
    
#Remember to:  
    #- Change the name of the seedlist. Dont call it .json. It will be appended. The seedlist must be in the rootfolder
    #- Adjust the start and endtime, and limits
    #- Insert email address for notifications (remember to look in the spamfolder)

#prerequisites:
    #- Seedlist must be in a flat .json structure as a list ["FB_ID_1"," FB_ID_2", etc]

import sys
import smtplib
import time
import json
import os

try:
    import facebook
except:
    print('Facebook-sdk not installed. Installing...')
    install_facebook()

import facebook

############Uncomment or create a new seedlist_name without .json############

#seedlist_name='missing_missing_DK-FB-Page-Race-PoliticiansSnowball_01-01-2012_31-12-2013_01-01-2012_31-12-2013'
seedlist_name='test'

############Insert appID and secret############

appID= '1056141877822551' #Backscatter original app
appsecret='ea9c62c22fed73bbf5aedba6b2b7843c'##Backscatter original app


############Adjust limits############

postlimit_start=20 #set to 80
reactionlimit='600' #max 1000
commentlimit= '40' #max 100

############Adjust start and end dates###########
startdate='01-01-2012'
enddate='31-12-2013'

############Mail setup###########
mail_address_1=""
mail_address_2=""
mails = [mail_address_1, mail_address_2]#'recipient@mailservice.com'
gmail_sender = 'motherscraper@gmail.com'
gmail_passwd = 'scrapeyourmother'
email_notifications=1 #1 for notifications and 0 for none



############Other initial definitions##########
seedlist=seedlist_name+'.json'
#seedlist=seedlist_name+'.json'
sleep_time=0.1
graph = facebook.GraphAPI(access_token=appID+'|'+appsecret, version='2.7')
start_time_total=time.time()


##########Create harvest folder##########
if not os.path.exists(seedlist_name):
    os.makedirs(seedlist_name)
    
##########Create json for skipped IDs#############    

missing='missing_'+seedlist_name+'_'+startdate+'_'+enddate+'.json'

if not os.path.exists(missing):
    with open(missing, 'w') as outfile:
        json.dump([], outfile, ensure_ascii=False)

        
def send_mail(TO, SUBJECT, TEXT):


    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(gmail_sender, gmail_passwd)

    BODY = '\r\n'.join(['To: %s' % TO,
                        'From: %s' % gmail_sender,
                        'Subject: %s' % SUBJECT,
                        '', TEXT])

    try:
        server.sendmail(gmail_sender, [TO], BODY)
        print('email sent to '+TO)
    except:
        print ('error sending mail')

    server.quit()        

def install_facebook():
    !{sys.executable} -m pip install facebook-sdk
    print('Facebook-sdk installed. Importing...')
    import facebook
    print('Facebook imported')
        
def skip_id(missing_data):
    with open(missing) as f:
        data=json.load(f)
    data.append(missing_data)
    with open(missing, 'w') as f:
        json.dump(data, f)


def paging_postreactions(): #comment_reactions or post_reactions or commentcomment_reactions
    connection_r=1
    reactionpaging_token=reactions['paging']
    while 'next' in reactionpaging_token and connection_r:
        reactionpaging_token=reactionpaging_token['next']
        reactionpaging_token = reactionpaging_token.split('/')
        reactionpaging_token = '/' + reactionpaging_token[4] + '/' + reactionpaging_token[5]
        reactionpaging_data=graph.get_object(id=reactionpaging_token)
        try:
            reactionpaging_data=graph.get_object(id=reactionpaging_token)
            for reaction in reactionpaging_data['data']:
                comment_reactions.append(reaction)
            reactionpaging_token=reactionpaging_data['paging']
            time.sleep(sleep_time)
        except:
            print('Problems with comments comments reactions.. Skipping ID')
            connection_r=0
            continue



def paging_commentreactions(): #comment_reactions or post_reactions or commentcomment_reactions
    connection_cr=1
    reactionpaging_token=reactions['paging']
    while 'next' in reactionpaging_token and connection_cr:
        reactionpaging_token=reactionpaging_token['next']
        reactionpaging_token = reactionpaging_token.split('/')
        reactionpaging_token = '/' + reactionpaging_token[4] + '/' + reactionpaging_token[5]
        try:
            reactionpaging_data=graph.get_object(id=reactionpaging_token)
            for reaction in reactionpaging_data['data']:
                comment_reactions.append(reaction)
            reactionpaging_token=reactionpaging_data['paging']
            time.sleep(sleep_time)
        except:
            print('Problems with comments reactions.. Skipping ID')
            connection_cr=0
            continue

def paging_commentcommentreactions(): #comment_reactions or post_reactions or commentcomment_reactions
    connection_ccr=1
    reactionpaging_token=reactions['paging']
    print("paging comment comment reaction")
    while 'next' in reactionpaging_token and connection_ccr:
        reactionpaging_token=reactionpaging_token['next']
        reactionpaging_token = reactionpaging_token.split('/')
        reactionpaging_token = '/' + reactionpaging_token[4] + '/' + reactionpaging_token[5]
        try:
            reactionpaging_data=graph.get_object(id=reactionpaging_token)
            for reaction in reactionpaging_data['data']:
                commentcomment_reactions.append(reaction)
            reactionpaging_token=reactionpaging_data['paging']
            time.sleep(sleep_time)
        except:
            print('Problems with comments comments reactions.. Skipping ID')
            connection_ccr=0
            continue


def paging_commentcomment(): 
    connection_commentcomment=1
    commentcommentpaging_token=commentcomments['paging']
    while 'next' in commentcommentpaging_token and connection_commentcomment:
        commentcommentpaging_token=commentcommentpaging_token['next']
        commentcommentpaging_token = commentcommentpaging_token.split('/')
        commentcommentpaging_token = '/' + commentcommentpaging_token[4] + '/' + commentcommentpaging_token[5]
        try:
            
            commentcommentpaging_data=graph.get_object(id=commentcommentpaging_token)
            for commentcomment in commentcommentpaging_data['data']:
                commentcomment_time = commentcomment['created_time']
                commentcomment_id = commentcomment['id']
                commentcomment_author = commentcomment['from']
                if 'message' in commentcomment:
                    commentcomment_message = commentcomment['message']
                else:
                    commentcomment_message = ''

                #iteratae over commentcomment reactions
                commentcomment_reactions=[]
                if 'reactions' in commentcomment:
                    reactions = commentcomment['reactions']
                    for reaction in reactions['data']:
                        commentcomment_reactions.append(reaction)
                    paging_commentcommentreactions()


                COMMENTCOMMENTDIC = {'commentcomment_time':commentcomment_time,'commentcomment_id':commentcomment_id,'commentcomment_message':commentcomment_message,'commentcomment_author':commentcomment_author,'commentcomment_reactions':commentcomment_reactions}
                commentcomments_data.append(COMMENTCOMMENTDIC)
            time.sleep(sleep_time)
            commentcommentpaging_token=commentcommentpaging_data['paging']
        except:
            print('Problems with comment paging.. Skipping ID')
            connection_commentcomment=0
            continue
       

            
def paging_comment(): #commentcomment or comment
    connection_comment=1
    y=1
    commentpaging_token=comments['paging']
    while 'next' in commentpaging_token and y<11 and connection_comment:
        commentpaging_token=commentpaging_token['next']
        commentpaging_token = commentpaging_token.split('/')
        commentpaging_token = '/' + commentpaging_token[4] + '/' + commentpaging_token[5]
        try:
            
            commentpaging_data=graph.get_object(id=commentpaging_token)
            for comment in commentpaging_data['data']:

                comment_time = comment['created_time']
                comment_id = comment['id']
                comment_author = comment['from']
                if 'message' in comment:
                    comment_message = comment['message']
                else:
                    comment_message = ''
                comment_reactions=[]
                if 'reactions' in comment:
                        reactions = comment['reactions'] 
                        for reaction in reactions['data']:
                            comment_reactions.append(reaction)
                        paging_commentreactions()
                commentcomments_data=[]
                if 'comment' in comment: 
                    commentcomment=comment['comments']
                    print(commentcomment)

                    paging_commentcomment()
                COMMENTDIC = {'comment_time':comment_time,'comment_id':comment_id,'comment_message':comment_message,'comment_author':comment_author,'comment_reactions':comment_reactions,'commentcomments':commentcomments_data}
                comments_data.append(COMMENTDIC)
            time.sleep(sleep_time)
            commentpaging_token=commentpaging_data['paging']
            y=y+1
        except:
            print('Problems with comment paging.. Skipping ID')
            connection_comment=0
            continue


x=0
data = json.load(open(seedlist))
data=set(data)
data=list(data)
all_FB_ID=len(data)
now = time.strftime("%H:%M", time.localtime(time.time()))
if all_FB_ID>60:
    estimated_duration=str(round((all_FB_ID/60)/3,0))+' hours' 
else:
    estimated_duration=str(round((all_FB_ID/3),0))+' minutes'
    
if email_notifications:
    for mail in mails:
        send_mail(mail, "Motherscraper started "+str(now), "Motherscraper started scraping from: "+seedlist+". Estimated duration: "+estimated_duration)
else:
    print('Email not send')
try:
    while x<all_FB_ID:
       # data = json.load(open(seedlist))
        start_time=time.time()
        FB_ID=data[x]
        print('Scraping '+FB_ID+' page '+str(x+1)+' of '+ str(all_FB_ID))
        postlimit=postlimit_start

        filepath=seedlist_name+'/'+FB_ID+'_'+startdate+'_to_'+enddate+'.json'
        with open(filepath, 'w') as outfile:
            json.dump([], outfile, ensure_ascii=False)

        post_count=0
        try: 
            print('Fetching data')
            feed_data = graph.get_object(FB_ID+'/feed?fields=from,created_time,link,picture,message,story,scheduled_publish_time,reactions.limit('+reactionlimit+'){id,type},comments.limit('+commentlimit+'){comments.limit('+commentlimit+'){from,created_time,message,id,reactions.limit('+reactionlimit+')},from,created_time,message,id,reactions.limit('+reactionlimit+')}&limit='+str(postlimit)+'&since='+startdate+'&until='+enddate)


        except:
            print('Problem with GRAPH GET. Lowering limit')
            print('Limit was: '+str(postlimit))
            postlimit=postlimit-10
            print('Limit is now: '+str(postlimit))
            try:
                feed_data = graph.get_object(FB_ID+'/feed?fields=from,created_time,link,picture,message,story,scheduled_publish_time,reactions.limit('+reactionlimit+'){id,type},comments.limit('+commentlimit+'){comments.limit('+commentlimit+'){from,created_time,message,id,reactions.limit('+reactionlimit+')},from,created_time,message,id,reactions.limit('+reactionlimit+')}&limit='+str(postlimit)+'&since='+startdate+'&until='+enddate)
            except:
                print('Something went wrong, skipping '+FB_ID+'. ID was saved in '+missing)
                x=x+1
                skip_id(FB_ID)
                continue
        if not feed_data['data']:
            print('Feed data empty. Skipping ID')
            x=x+1
            continue
        else:
            x=x+1
        DATA = []

        for post in feed_data['data']:
            post_time = post['created_time']
            post_id = post['id']
            post_author = post['from']
            print('PostID: '+ str(post_id)+' from '+post_time)
            if 'message' in post:
                post_message = post['message']
            else:
                post_message = ''
            if 'picture' in post:
                post_picture = post['picture']
            else:
                post_picture = ''
            if 'link' in post:
                post_link = post['link']
            else:
                post_link = ''

            #iterate over post reactions
            post_reactions = []
            if 'reactions' in post:
                reactions = post['reactions']
                for reaction in reactions['data']:
                    post_reactions.append(reaction)
                paging_postreactions()

            #iterate over comments
            comments_data = []
            if 'comments' in post:
                comments = post['comments']
                for comment in comments['data']: 
                    comment_time = comment['created_time']
                    comment_id = comment['id']
                    comment_author = comment['from']
                    if 'message' in comment:
                        comment_message = comment['message']
                    else:
                        comment_message = ''

                    #iterate over comment reactions
                    comment_reactions = []
                    if 'reactions' in comment:
                        reactions = comment['reactions']
                        for reaction in reactions['data']:
                            comment_reactions.append(reaction)
                        paging_commentreactions()

                    #iterate over comments on comments
                    commentcomments_data = []
                    if 'comments' in comment:
                        commentcomments = comment['comments']
                        for commentcomment in commentcomments['data']:
                            commentcomment_time = commentcomment['created_time']
                            commentcomment_id = commentcomment['id']
                            commentcomment_author = commentcomment['from']
                            if 'message' in commentcomment:
                                commentcomment_message = commentcomment['message']
                            else:
                                commentcomment_message = ''

                            #iteratae over commentcomment reactions
                            commentcomment_reactions = []
                            if 'reactions' in commentcomments:
                                reactions = commentcomment['reactions']
                                for reaction in reactions['data']:
                                    commentcomment_reactions.append(reaction)
                                paging_commentcommentreactions()

                            COMMENTCOMMENTDIC = {'commentcomment_time':commentcomment_time,'commentcomment_id':commentcomment_id,'commentcomment_message':commentcomment_message,'commentcomment_author':commentcomment_author,'commentcomment_reactions':commentcomment_reactions}
                            commentcomments_data.append(COMMENTCOMMENTDIC)
                        paging_commentcomment()



                    COMMENTDIC = {'comment_time':comment_time,'comment_id':comment_id,'comment_message':comment_message,'comment_author':comment_author,'comment_reactions':comment_reactions,'commentcomments':commentcomments_data}
                    comments_data.append(COMMENTDIC)
                paging_comment()
            POSTDATA = {'post_id':post_id,'post_time':post_time,'post_message':post_message,'post_author':post_author,'post_picture':post_picture,'post_link':post_link,'post_reactions':post_reactions,'comments':comments_data}

            DATA.append(POSTDATA)


            post_count=post_count+1
            print('Posts scraped '+str(post_count))

        print("Finished first post page.")
        if 'next' in feed_data['paging']:
            print('Paging posts for '+FB_ID)
        connection_page=1


        #########################POST PAGING############################
        try:
            while 'next' in feed_data['paging'] and connection_page:
                post_paging_token = feed_data['paging']['next']
                post_paging_token = post_paging_token.split('/')
                post_paging_token = '/' + post_paging_token[4] + '/' + post_paging_token[5]
                try: 
                    print('Fetching data')
                    feed_data = graph.get_object(id=post_paging_token)

                except:
                    print('Problem with GRAPH GET. Lowering limit')


                    try:
                        time.sleep(1)
                        limit=postlimit-10
                        print('Postlimit was: '+str(postlimit)+' is now: '+str(limit))
                        post_paging_token = post_paging_token.split('&')
                        new_post_paging_token=post_paging_token[0] + '&' + post_paging_token[1] + '&' + post_paging_token[2]+ '&' + post_paging_token[3]+ '&limit='+str(limit)+'&' + post_paging_token[5]            
                        feed_data = graph.get_object(id=new_post_paging_token)

                    except:    
                        print('Still problems with GRAPH GET. Lowering limit again') 
                        try:
                            time.sleep(1)
                            limit_2=10
                            print('Postlimit was: '+str(limit)+' is now: '+str(limit_2))
                            post_paging_token = post_paging_token.split('&')
                            new_post_paging_token=post_paging_token[0] + '&' + post_paging_token[1] + '&' + post_paging_token[2]+ '&' + post_paging_token[3]+ '&limit='+str(limit_2)+'&' + post_paging_token[5]
                            feed_data = graph.get_object(id=new_post_paging_token)
                        except:
                            print('Could not get paging data. Stopping paging for '+FB_ID+'. ID was saved in missing.json.')
                            skip_id(FB_ID)
                            connection_page=0
                            continue

                for post in feed_data['data']:
                    post_time = post['created_time']
                    post_id = post['id']
                    post_author = post['from']
                    print('PostID: '+ str(post_id)+' from '+post_time)

                    if 'message' in post:
                        post_message = post['message']
                    else:
                        post_message = ''
                    if 'picture' in post:
                        post_picture = post['picture']
                    else:
                        post_picture = ''
                    if 'link' in post:
                        post_link = post['link']
                    else:
                        post_link = ''

                    #iterate over post reactions
                    post_reactions = []
                    if 'reactions' in post:
                        reactions = post['reactions']
                        for reaction in reactions['data']:
                            post_reactions.append(reaction)
                        paging_postreactions()

                    #iterate over comments
                    comments_data = []
                    if 'comments' in post:
                        comments = post['comments']
                        for comment in comments['data']: 
                            comment_time = comment['created_time']
                            comment_id = comment['id']
                            comment_author = comment['from']
                            if 'message' in comment:
                                comment_message = comment['message']
                            else:
                                comment_message = ''

                            #iterate over comment reactions
                            comment_reactions = []
                            if 'reactions' in comment:
                                reactions = comment['reactions']
                                for reaction in reactions['data']:
                                    comment_reactions.append(reaction)
                                paging_commentreactions()

                            #iterate over comments on comments
                            commentcomments_data = []
                            if 'comments' in comment:

                                commentcomments = comment['comments']
                                for commentcomment in commentcomments['data']:
                                    commentcomment_time = commentcomment['created_time']
                                    commentcomment_id = commentcomment['id']
                                    commentcomment_author = commentcomment['from']
                                    if 'message' in commentcomment:
                                        commentcomment_message = commentcomment['message']
                                    else:
                                        commentcomment_message = ''

                                    #iteratae over commentcomment reactions
                                    commentcomment_reactions = []
                                    if 'reactions' in commentcomments:
                                        reactions = commentcomment['reactions']
                                        for reaction in reactions['data']:
                                            commentcomment_reactions.append(reaction)
                                        paging_commentcommentreactions()

                                    COMMENTCOMMENTDIC = {'commentcomment_time':commentcomment_time,'commentcomment_id':commentcomment_id,'commentcomment_message':commentcomment_message,'commentcomment_author':commentcomment_author,'commentcomment_reactions':commentcomment_reactions}
                                    commentcomments_data.append(COMMENTCOMMENTDIC)
                                paging_commentcomment()



                            COMMENTDIC = {'comment_time':comment_time,'comment_id':comment_id,'comment_message':comment_message,'comment_author':comment_author,'comment_reactions':comment_reactions,'commentcomments':commentcomments_data}
                            comments_data.append(COMMENTDIC)
                        paging_comment()
                    POSTDATA = {'post_id':post_id,'post_time':post_time,'post_message':post_message,'post_author':post_author,'post_picture':post_picture,'post_link':post_link,'post_reactions':post_reactions,'comments':comments_data}

                    DATA.append(POSTDATA)    

                    post_count=post_count+1
                    print('Posts scraped '+str(post_count)) 
        except:
            skip_id(FB_ID)
            with open(filepath, 'w') as f:
                json.dump(DATA, f)
            continue
        with open(filepath, 'w') as f:
            json.dump(DATA, f)

        end_time=time.time()
        duration=end_time-start_time 
        now = time.strftime("%H:%M", time.localtime(time.time()))
        print(str(FB_ID)+' with '+str(post_count)+' posts scraped in: '+str(round(duration,2))+' seconds. Time is: '+str(now))

    end_time_total=time.time()
    duration_total=round(((end_time_total-start_time_total)/60),0)
    now = time.strftime("%H:%M", time.localtime(time.time()))

    print('Scraping of '+seedlist+ ' ended at '+str(now)+' after '+str(duration_total)+' minutes')
    if email_notifications:
        for mail in mails:
            send_mail(mail,"Motherscraper finished "+str(now), "Motherscraper successfully finished scraping "+seedlist+" in "+ str(duration_total)+" minutes")
except Exception as e: 
    print('Problem in the script: '+str(e)+' \nPlease edit the seedlist to start at '+str(FB_ID)+' and restart the script')
    if email_notifications:
        for mail in mails:
            send_mail(mail,"Motherscraper has unexpectedly stopped "+str(now), "Motherscraper scraping from "+seedlist+" has unexspectedly stopped at "+str(FB_ID))

print('Scraping done')


error sending mail
error sending mail
Scraping 85891659096 page 1 of 3
Fetching data
PostID: 85891659096_10152061499867225 from 2013-12-26T19:52:47+0000
Posts scraped 1
PostID: 85891659096_10152506479204097 from 2013-12-26T08:00:01+0000
Posts scraped 2
PostID: 85891659096_10152505333799097 from 2013-12-25T22:12:45+0000
Posts scraped 3
PostID: 85891659096_10152491231359097 from 2013-12-20T08:51:01+0000
Posts scraped 4
PostID: 85891659096_10152487119629097 from 2013-12-18T14:00:02+0000
Posts scraped 5
PostID: 85891659096_10152486722009097 from 2013-12-18T08:00:01+0000
Posts scraped 6
PostID: 85891659096_10152484827859097 from 2013-12-17T14:00:01+0000
Posts scraped 7
PostID: 85891659096_10152484379994097 from 2013-12-17T08:00:01+0000
Posts scraped 8
PostID: 85891659096_10152482402789097 from 2013-12-16T14:00:01+0000
Posts scraped 9
PostID: 85891659096_10152481941124097 from 2013-12-16T08:00:01+0000
Posts scraped 10
PostID: 85891659096_10152479764164097 from 2013-12-15T14:00:01+0000
Posts 

PostID: 85891659096_10152314863964097 from 2013-10-20T17:00:33+0000
Posts scraped 101
PostID: 85891659096_10202435517755114 from 2013-10-20T07:17:16+0000
Posts scraped 102
PostID: 85891659096_10152310672944097 from 2013-10-19T07:08:45+0000
Posts scraped 103
PostID: 85891659096_10152306519929097 from 2013-10-17T16:52:12+0000
Posts scraped 104
PostID: 85891659096_10152306503679097 from 2013-10-17T16:42:14+0000
Posts scraped 105
PostID: 85891659096_10152306497369097 from 2013-10-17T16:38:26+0000
Posts scraped 106
PostID: 85891659096_10152303532804097 from 2013-10-16T08:11:03+0000
Posts scraped 107
PostID: 85891659096_10152303519844097 from 2013-10-16T07:58:05+0000
Posts scraped 108
PostID: 85891659096_10152303429099097 from 2013-10-16T06:27:01+0000
Posts scraped 109
PostID: 85891659096_10152303362184097 from 2013-10-16T05:32:27+0000
Posts scraped 110
PostID: 85891659096_10152301161974097 from 2013-10-15T10:36:41+0000
Posts scraped 111
PostID: 85891659096_10152301036729097 from 2013-10-15T

PostID: 85891659096_10151857364739097 from 2013-04-03T11:44:15+0000
Posts scraped 201
PostID: 85891659096_154722114688734 from 2013-04-01T22:29:45+0000
Posts scraped 202
PostID: 85891659096_356264387817171 from 2013-03-28T09:21:37+0000
Posts scraped 203
PostID: 85891659096_361162360654408 from 2013-03-27T09:47:45+0000
Posts scraped 204
PostID: 85891659096_479812895407402 from 2013-03-25T11:10:46+0000
Posts scraped 205
PostID: 85891659096_547347235285423 from 2013-03-22T20:41:05+0000
Posts scraped 206
PostID: 85891659096_110219815815100 from 2013-03-21T18:48:08+0000
Posts scraped 207
PostID: 85891659096_398529736911692 from 2013-03-20T19:27:08+0000
Posts scraped 208
PostID: 85891659096_10151831489639097 from 2013-03-20T18:50:33+0000
Posts scraped 209
PostID: 85891659096_431656883585166 from 2013-03-14T20:49:26+0000
Posts scraped 210
PostID: 85891659096_342399785861734 from 2013-03-07T12:23:17+0000
Posts scraped 211
PostID: 85891659096_116756285177729 from 2013-03-06T21:32:54+0000
Posts 

PostID: 85891659096_195854930549666 from 2012-10-15T09:05:33+0000
Posts scraped 301
PostID: 85891659096_289625067819247 from 2012-10-11T18:10:30+0000
Posts scraped 302
PostID: 85891659096_10151475548124097 from 2012-10-07T15:54:39+0000
Posts scraped 303
PostID: 85891659096_289704234469712 from 2012-10-06T08:42:15+0000
Posts scraped 304
PostID: 85891659096_267062893396454 from 2012-10-02T18:44:53+0000
Posts scraped 305
PostID: 85891659096_449342261771186 from 2012-09-27T19:13:21+0000
Posts scraped 306
PostID: 85891659096_10151459052234097 from 2012-09-27T16:05:11+0000
Posts scraped 307
PostID: 85891659096_10151457856974097 from 2012-09-26T20:14:01+0000
Posts scraped 308
PostID: 85891659096_420676231322282 from 2012-09-26T12:36:46+0000
Posts scraped 309
PostID: 85891659096_10151455229359097 from 2012-09-25T05:36:31+0000
Posts scraped 310
PostID: 85891659096_10151452826829097 from 2012-09-23T20:44:37+0000
Posts scraped 311
PostID: 85891659096_421452137912224 from 2012-09-23T08:13:40+0000
